# Part 3: Training and evaluating an MLP classifier with Torchbearer

In [1]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

## Introducing Torchbearer
You've now got to a stage where you've successfully implemented, trained and evaluated a neural network in PyTorch. You will have noticed that whilst defining the model was done in very few lines of code, that you actually had to do quite a lot of work to train and evaluate the model. Whilst the ability to have complete control over training and evaluation is useful (you'll see examples in later labs, and for the coursework you might come across situations where this is an absolute necessity), it can become rather tedious if you just want to perform a standard training and evaluation run. 

The [Torchbearer](https://github.com/ecs-vlc/torchbearer) library, written and maintained by members of the VLC research group in ECS, can help. Torchbearer is a model training and evaluation library that is designed to massively reduce the amount of code you need to write whilst still allowing full control over the process. 

The following code just reproduces the baseline MLP model implementation and loads data as we did in part 2:

In [2]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

# fix random seed for reproducibility
seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
import numpy as np
np.random.seed(seed)

# flatten 28*28 images to a 784 vector for each image
transform = transforms.Compose([
    transforms.ToTensor(),  # convert to tensor
    transforms.Lambda(lambda x: x.view(-1))  # flatten into vector
])

# load data
trainset = MNIST(".", train=True, download=True, transform=transform)
testset = MNIST(".", train=False, download=True, transform=transform)

# create data loaders
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=True)

# define baseline model
class BaselineModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(BaselineModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        if not self.training:
            out = F.softmax(out, dim=1)
        return out

# build the model
model = BaselineModel(784, 784, 10)

In [2]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

# fix random seed for reproducibility
seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
import numpy as np
np.random.seed(seed)

# flatten 28*28 images to a 784 vector for each image
transform = transforms.Compose([
    transforms.ToTensor(),  # convert to tensor
    transforms.Lambda(lambda x: x.view(-1))  # flatten into vector
])

# load data
trainset = MNIST(".", train=True, download=True, transform=transform)
testset = MNIST(".", train=False, download=True, transform=transform)

# create data loaders
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=True)

# define baseline model
class BaselineModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(BaselineModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        if not self.training:
            out = F.softmax(out, dim=1)
        return out
    
# build the model
model = BaselineModel(784, 60000, 10)

We can use the torchbearer `Trial` class to do all the hard work in training and evaluating for us:

In [3]:
import torchbearer

# define the loss function and the optimiser
loss_function = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters())

# Construct a trial object with the model, optimiser and loss.
# Also specify metrics we wish to compute.
trial = torchbearer.Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy'])

# Provide the data to the trial
trial.with_generators(trainloader, test_generator=testloader)

# Run 10 epochs of training
trial.run(epochs=10)

# test the performance
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)

KeyboardInterrupt: 

In [4]:
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)



{'test_loss': 1.5319161415100098, 'test_acc': 0.9502000212669373}


You can see that training and evaluating the model prints out much more informative information as the process runs - this is particularly useful for training big models as it enables you to keep an eye on progress. You should see the accuracy matches the one from the previous part of the lab.

Take some time to have a look over the [Torchbearer documentation](https://torchbearer.readthedocs.io). Once you've had a look, __use the code block below to train the model (newly initialised) with a plain Stochastic Gradient Descent optimiser with a learning rate of 0.01 (keep all other SGD parameters at their default values). In addition to computing the loss and accuracy metrics, use Torchbearer to also compute the top-5 accuracy metric which was made popular by the ImageNet challenge.__

In [5]:
model = BaselineModel(784, 784, 10)

# YOUR CODE HERE

# define the loss function and the optimiser
loss_function = nn.CrossEntropyLoss()
optimiser = optim.SGD(model.parameters(),lr=0.01)
# Construct a trial object with the model, optimiser and loss.
# Also specify metrics we wish to compute.
trial = torchbearer.Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy','top_5_acc'])
# Provide the data to the trial
trial.with_generators(trainloader, test_generator=testloader)


trial.run(epochs=10)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)


{'test_loss': 1.6033602952957153, 'test_acc': 0.9176999926567078, 'test_top_5_acc': 0.9945999979972839}


In [ ]:
assert 'test_top_5_acc' in results
